# 底層實作01: 從零打造 Naive Bayes 分類器

**課程**: iSpan Python NLP Cookbooks v2
**章節**: 底層實作系列
**版本**: v1.0
**更新日期**: 2025-10-17

---

## 📚 本節學習目標

1. 從數學原理推導 Naive Bayes 算法
2. 理解貝葉斯定理與條件獨立假設
3. 從零實作 Multinomial Naive Bayes
4. 掌握 Laplace Smoothing 技巧
5. 與 scikit-learn 版本對比驗證

---

## 1. 貝葉斯定理數學推導

### 1.1 貝葉斯定理 (Bayes' Theorem)

**定義**: 描述在已知某些條件下,事件發生的機率

```
P(A|B) = P(B|A) * P(A) / P(B)

其中:
- P(A|B): 後驗機率 (Posterior) - 在 B 發生條件下 A 的機率
- P(B|A): 似然 (Likelihood) - 在 A 發生條件下 B 的機率
- P(A):   先驗機率 (Prior) - A 發生的機率
- P(B):   證據 (Evidence) - B 發生的機率
```

### 1.2 應用到文本分類

**目標**: 給定文本 D,預測類別 C

```
P(C|D) = P(D|C) * P(C) / P(D)

簡化 (因為 P(D) 對所有類別相同):
C_pred = argmax_C [ P(D|C) * P(C) ]

展開文本 D = {w1, w2, ..., wn} (詞序列):
C_pred = argmax_C [ P(w1, w2, ..., wn | C) * P(C) ]
```

### 1.3 條件獨立假設 (Naive Assumption)

**假設**: 每個詞彙在類別 C 下**相互獨立**

```
P(w1, w2, ..., wn | C) = P(w1|C) * P(w2|C) * ... * P(wn|C)

簡化為:
P(w1, w2, ..., wn | C) = ∏ P(wi|C)  (i=1 to n)

最終分類公式:
C_pred = argmax_C [ P(C) * ∏ P(wi|C) ]

取對數避免下溢 (underflow):
C_pred = argmax_C [ log P(C) + Σ log P(wi|C) ]
```

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict

# 視覺化貝葉斯定理
def visualize_bayes_theorem():
    # 模擬數據: 垃圾郵件 vs 正常郵件
    categories = ['Spam', 'Ham']
    priors = [0.3, 0.7]  # P(Spam)=0.3, P(Ham)=0.7
    
    # 似然: P(包含"free"|類別)
    likelihoods = [0.8, 0.2]  # Spam中80%有"free", Ham中20%有"free"
    
    # 計算後驗機率 P(類別|"free")
    evidence = sum(p * l for p, l in zip(priors, likelihoods))
    posteriors = [(l * p) / evidence for p, l in zip(priors, likelihoods)]
    
    # 繪圖
    fig, axes = plt.subplots(1, 3, figsize=(15, 4))
    
    axes[0].bar(categories, priors, color=['#ff6b6b', '#4ecdc4'])
    axes[0].set_title('Prior P(C)', fontsize=14)
    axes[0].set_ylabel('Probability')
    
    axes[1].bar(categories, likelihoods, color=['#ff6b6b', '#4ecdc4'])
    axes[1].set_title('Likelihood P("free"|C)', fontsize=14)
    axes[1].set_ylabel('Probability')
    
    axes[2].bar(categories, posteriors, color=['#ff6b6b', '#4ecdc4'])
    axes[2].set_title('Posterior P(C|"free")', fontsize=14)
    axes[2].set_ylabel('Probability')
    
    plt.tight_layout()
    plt.show()
    
    print("貝葉斯更新過程:")
    print(f"Prior:     Spam={priors[0]:.2f}, Ham={priors[1]:.2f}")
    print(f"Posterior: Spam={posteriors[0]:.2f}, Ham={posteriors[1]:.2f}")
    print(f"\n結論: 看到'free'後,Spam機率從 {priors[0]:.0%} 上升到 {posteriors[0]:.0%}")

visualize_bayes_theorem()

---

## 2. Multinomial Naive Bayes 實作

### 2.1 算法步驟

**訓練階段**:
1. 計算每個類別的先驗機率 P(C)
2. 計算每個詞在每個類別中的條件機率 P(w|C)
3. 應用 Laplace Smoothing 避免零機率

**預測階段**:
1. 對新文本計算 log P(C) + Σ log P(wi|C)
2. 選擇分數最高的類別

### 2.2 從零實作

In [ ]:
class NaiveBayesClassifier:
    def __init__(self, alpha=1.0):
        """
        Parameters:
        -----------
        alpha : float
            Laplace smoothing 參數 (加法平滑)
        """
        self.alpha = alpha
        self.class_log_prior = {}     # log P(C)
        self.feature_log_prob = {}    # log P(w|C)
        self.classes = None
        self.vocabulary = set()
    
    def fit(self, X, y):
        """
        訓練模型
        
        Parameters:
        -----------
        X : list of str
            訓練文本列表
        y : list of str/int
            訓練標籤列表
        """
        n_samples = len(X)
        self.classes = np.unique(y)
        
        # 建立詞彙表
        for text in X:
            self.vocabulary.update(text.lower().split())
        
        vocab_size = len(self.vocabulary)
        
        # 對每個類別計算統計量
        for c in self.classes:
            # 獲取該類別的所有文本
            c_texts = [X[i] for i in range(n_samples) if y[i] == c]
            n_c = len(c_texts)
            
            # 計算先驗機率 P(C)
            self.class_log_prior[c] = np.log(n_c / n_samples)
            
            # 統計詞頻
            word_counts = defaultdict(int)
            total_words = 0
            
            for text in c_texts:
                words = text.lower().split()
                for word in words:
                    word_counts[word] += 1
                    total_words += 1
            
            # 計算條件機率 P(w|C) with Laplace Smoothing
            self.feature_log_prob[c] = {}
            for word in self.vocabulary:
                count = word_counts[word]
                # Laplace Smoothing: (count + alpha) / (total + alpha * vocab_size)
                prob = (count + self.alpha) / (total_words + self.alpha * vocab_size)
                self.feature_log_prob[c][word] = np.log(prob)
        
        return self
    
    def predict(self, X):
        """
        預測新文本
        
        Parameters:
        -----------
        X : list of str
            待預測文本列表
        
        Returns:
        --------
        predictions : list
            預測的類別列表
        """
        predictions = []
        
        for text in X:
            words = text.lower().split()
            class_scores = {}
            
            for c in self.classes:
                # 初始化為 log P(C)
                score = self.class_log_prior[c]
                
                # 累加 Σ log P(wi|C)
                for word in words:
                    if word in self.vocabulary:
                        score += self.feature_log_prob[c][word]
                
                class_scores[c] = score
            
            # 選擇分數最高的類別
            predicted_class = max(class_scores, key=class_scores.get)
            predictions.append(predicted_class)
        
        return predictions
    
    def predict_proba(self, X):
        """
        預測機率
        """
        probabilities = []
        
        for text in X:
            words = text.lower().split()
            class_scores = {}
            
            for c in self.classes:
                score = self.class_log_prior[c]
                for word in words:
                    if word in self.vocabulary:
                        score += self.feature_log_prob[c][word]
                class_scores[c] = score
            
            # 轉換為機率 (softmax)
            scores = np.array(list(class_scores.values()))
            exp_scores = np.exp(scores - np.max(scores))  # 數值穩定
            probs = exp_scores / exp_scores.sum()
            
            probabilities.append(dict(zip(class_scores.keys(), probs)))
        
        return probabilities

# 測試實作
print("✅ Naive Bayes 分類器實作完成")

---

## 3. Laplace Smoothing 深入理解

### 3.1 零機率問題

**問題**: 如果訓練集中某個詞從未在某類別出現,則 P(w|C) = 0

```
訓練集:
Spam: "buy free now"
Ham:  "hello friend"

測試: "free hello"
P(Spam|"free hello") = P(Spam) * P(free|Spam) * P(hello|Spam)
                     = 0.5 * 0.33 * 0  ← hello 從未在 Spam 出現
                     = 0  ← 整個機率變 0!
```

### 3.2 Laplace Smoothing 解決方案

**公式**:
```
P(wi|C) = (count(wi, C) + α) / (count(C) + α * |V|)

其中:
- α: smoothing 參數 (通常為 1)
- |V|: 詞彙表大小
- count(wi, C): 詞 wi 在類別 C 中出現次數
- count(C): 類別 C 的總詞數
```

In [ ]:
# 演示 Laplace Smoothing 效果
def compare_smoothing():
    # 模擬數據
    word_counts = {'buy': 2, 'free': 3, 'now': 1}  # Spam 詞頻
    total_words = sum(word_counts.values())  # 6
    vocab_size = 10  # 假設詞彙表有 10 個詞
    
    test_word = 'hello'  # 訓練集中未出現的詞
    
    # 不使用 smoothing
    prob_no_smooth = word_counts.get(test_word, 0) / total_words
    
    # 使用 Laplace smoothing (α=1)
    prob_smooth = (word_counts.get(test_word, 0) + 1) / (total_words + vocab_size)
    
    print("Laplace Smoothing 效果對比:")
    print("="*50)
    print(f"詞彙: '{test_word}' (訓練集中未出現)")
    print(f"\n不使用 Smoothing:")
    print(f"  P('{test_word}'|Spam) = {word_counts.get(test_word, 0)}/{total_words} = {prob_no_smooth}")
    print(f"\n使用 Laplace Smoothing (α=1):")
    print(f"  P('{test_word}'|Spam) = ({word_counts.get(test_word, 0)}+1)/({total_words}+{vocab_size}) = {prob_smooth:.4f}")
    print(f"\n結果: Smoothing 避免了零機率問題!")

compare_smoothing()

---

## 4. 實戰案例: 垃圾郵件分類

### 4.1 準備數據集

In [ ]:
# 垃圾郵件分類數據集
spam_emails = [
    "free money now",
    "win prizes click here",
    "buy cheap products",
    "get rich quick",
    "earn money fast",
    "free gift click now",
    "win big prizes today",
    "make money online"
]

ham_emails = [
    "hello friend how are you",
    "meeting tomorrow at noon",
    "please review the document",
    "thanks for your help",
    "see you next week",
    "happy birthday dear friend",
    "project deadline is friday",
    "let me know your thoughts"
]

# 合併數據
X_train = spam_emails + ham_emails
y_train = ['spam'] * len(spam_emails) + ['ham'] * len(ham_emails)

print(f"訓練集大小: {len(X_train)}")
print(f"Spam 樣本: {y_train.count('spam')}")
print(f"Ham 樣本:  {y_train.count('ham')}")

### 4.2 訓練與預測

In [ ]:
# 訓練模型
nb_classifier = NaiveBayesClassifier(alpha=1.0)
nb_classifier.fit(X_train, y_train)

# 測試數據
X_test = [
    "free money click here",      # 應該是 spam
    "meeting next week",           # 應該是 ham
    "win big prizes now",          # 應該是 spam
    "thanks for the document",     # 應該是 ham
    "buy free products",           # 應該是 spam
]

# 預測
predictions = nb_classifier.predict(X_test)
probabilities = nb_classifier.predict_proba(X_test)

# 顯示結果
print("\n預測結果:")
print("="*70)
for i, (text, pred, prob) in enumerate(zip(X_test, predictions, probabilities), 1):
    print(f"{i}. 文本: '{text}'")
    print(f"   預測: {pred.upper()}")
    print(f"   機率: Spam={prob['spam']:.2%}, Ham={prob['ham']:.2%}\n")

### 4.3 查看學習到的特徵

In [ ]:
# 找出每個類別最重要的詞
def get_top_features(classifier, top_n=5):
    for class_name in classifier.classes:
        # 獲取該類別的詞機率
        word_probs = classifier.feature_log_prob[class_name]
        
        # 排序 (log 機率越大越重要)
        sorted_words = sorted(word_probs.items(), key=lambda x: x[1], reverse=True)
        
        print(f"\n{class_name.upper()} 最重要的 {top_n} 個詞:")
        for word, log_prob in sorted_words[:top_n]:
            prob = np.exp(log_prob)
            print(f"  {word:15s} → P(w|{class_name}) = {prob:.4f}")

get_top_features(nb_classifier, top_n=5)

---

## 5. 與 scikit-learn 對比驗證

### 5.1 使用相同數據訓練 sklearn 版本

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# 建立 sklearn Pipeline
sklearn_pipeline = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('classifier', MultinomialNB(alpha=1.0))
])

# 訓練
sklearn_pipeline.fit(X_train, y_train)

# 預測
sklearn_predictions = sklearn_pipeline.predict(X_test)
sklearn_proba = sklearn_pipeline.predict_proba(X_test)

print("sklearn 預測結果:")
print("="*70)
classes = sklearn_pipeline.classes_
for i, (text, pred, proba) in enumerate(zip(X_test, sklearn_predictions, sklearn_proba), 1):
    print(f"{i}. 文本: '{text}'")
    print(f"   預測: {pred.upper()}")
    prob_dict = dict(zip(classes, proba))
    print(f"   機率: Spam={prob_dict.get('spam', 0):.2%}, Ham={prob_dict.get('ham', 0):.2%}\n")

### 5.2 結果對比分析

In [ ]:
import pandas as pd

# 對比預測結果
comparison_df = pd.DataFrame({
    '文本': X_test,
    '自製預測': predictions,
    'sklearn預測': sklearn_predictions,
    '結果一致': [p1 == p2 for p1, p2 in zip(predictions, sklearn_predictions)]
})

print("\n預測結果對比:")
print("="*80)
print(comparison_df.to_string(index=False))

# 計算一致率
accuracy = comparison_df['結果一致'].mean()
print(f"\n一致率: {accuracy:.0%}")

if accuracy == 1.0:
    print("✅ 完美! 自製版本與 sklearn 預測完全一致")
else:
    print("⚠️  存在差異,需檢查實作細節")

---

## 6. 效能分析與可視化

### 6.1 混淆矩陣

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

# 在訓練集上評估
train_predictions = nb_classifier.predict(X_train)

# 計算混淆矩陣
cm = confusion_matrix(y_train, train_predictions, labels=['spam', 'ham'])

# 繪製混淆矩陣
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Spam', 'Ham'],
            yticklabels=['Spam', 'Ham'])
plt.title('Confusion Matrix (Training Set)', fontsize=14)
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()

# 分類報告
print("\n分類報告 (訓練集):")
print(classification_report(y_train, train_predictions, target_names=['spam', 'ham']))

### 6.2 特徵重要性可視化

In [ ]:
# 比較 Spam vs Ham 的詞機率
def visualize_feature_importance(classifier, top_n=10):
    # 獲取共同詞彙
    spam_probs = classifier.feature_log_prob['spam']
    ham_probs = classifier.feature_log_prob['ham']
    
    # 計算差異 (log odds ratio)
    word_scores = {}
    for word in classifier.vocabulary:
        spam_prob = np.exp(spam_probs[word])
        ham_prob = np.exp(ham_probs[word])
        # 越正表示越像 Spam,越負表示越像 Ham
        score = np.log(spam_prob / ham_prob)
        word_scores[word] = score
    
    # 排序
    sorted_words = sorted(word_scores.items(), key=lambda x: abs(x[1]), reverse=True)
    
    # 取前 top_n
    words = [w for w, s in sorted_words[:top_n]]
    scores = [s for w, s in sorted_words[:top_n]]
    
    # 繪圖
    plt.figure(figsize=(12, 6))
    colors = ['#ff6b6b' if s > 0 else '#4ecdc4' for s in scores]
    plt.barh(words, scores, color=colors)
    plt.xlabel('Log Odds Ratio (Spam vs Ham)', fontsize=12)
    plt.title('Most Discriminative Words', fontsize=14)
    plt.axvline(x=0, color='black', linestyle='--', linewidth=0.8)
    
    # 添加圖例
    from matplotlib.patches import Patch
    legend_elements = [
        Patch(facecolor='#ff6b6b', label='Spam'),
        Patch(facecolor='#4ecdc4', label='Ham')
    ]
    plt.legend(handles=legend_elements, loc='best')
    
    plt.tight_layout()
    plt.show()

visualize_feature_importance(nb_classifier, top_n=15)

---

## 7. 進階: 處理真實數據集

### 7.1 使用 SMS Spam Collection 數據集

In [ ]:
# 如果有 SMS Spam 數據集
try:
    import pandas as pd
    from sklearn.model_selection import train_test_split
    
    # 嘗試載入數據
    # df = pd.read_csv('../../datasets/sms_spam/SMSSpamCollection', 
    #                  sep='\t', names=['label', 'message'])
    
    # 暫時使用模擬數據
    df = pd.DataFrame({
        'label': ['spam', 'ham'] * 50,
        'message': ['free money now'] * 50 + ['hello friend'] * 50
    })
    
    # 切分訓練集與測試集
    X = df['message'].tolist()
    y = df['label'].tolist()
    
    X_train_real, X_test_real, y_train_real, y_test_real = train_test_split(
        X, y, test_size=0.2, random_state=42
    )
    
    # 訓練
    nb_real = NaiveBayesClassifier(alpha=1.0)
    nb_real.fit(X_train_real, y_train_real)
    
    # 預測
    y_pred_real = nb_real.predict(X_test_real)
    
    # 評估
    from sklearn.metrics import accuracy_score, f1_score
    
    accuracy = accuracy_score(y_test_real, y_pred_real)
    f1 = f1_score(y_test_real, y_pred_real, pos_label='spam')
    
    print("真實數據集評估結果:")
    print(f"準確率: {accuracy:.2%}")
    print(f"F1 分數: {f1:.2%}")
    
except Exception as e:
    print(f"無法載入真實數據集: {e}")
    print("請確保數據集位於正確路徑")

---

## 8. 課後練習

### 練習 1: 實作 Bernoulli Naive Bayes

提示: 與 Multinomial 不同,Bernoulli 只考慮詞是否出現 (0/1),而非出現次數。

In [ ]:
# TODO: 實作 Bernoulli Naive Bayes
class BernoulliNaiveBayes:
    def __init__(self, alpha=1.0):
        self.alpha = alpha
        # TODO: 初始化其他必要參數
    
    def fit(self, X, y):
        # TODO: 實作訓練邏輯
        # 提示: P(w|C) = (出現該詞的文檔數 + α) / (類別 C 的文檔數 + 2α)
        pass
    
    def predict(self, X):
        # TODO: 實作預測邏輯
        pass

### 練習 2: 處理中文文本

使用 jieba 分詞,將 Naive Bayes 應用到中文垃圾簡訊分類。

In [ ]:
# TODO: 中文文本分類
import jieba

# 中文數據集範例
chinese_spam = [
    "恭喜您中獎了,請點擊連結領取",
    "免費贈送禮品,馬上領取"
]

chinese_ham = [
    "明天會議時間改到下午三點",
    "週末一起去吃飯吧"
]

# TODO: 使用 jieba 分詞 + Naive Bayes 分類

---

## 9. 本節總結

### ✅ 關鍵要點

1. **貝葉斯定理**: P(C|D) = P(D|C) * P(C) / P(D)
   - 後驗機率 = (似然 × 先驗) / 證據

2. **條件獨立假設**: P(w1, w2, ..., wn | C) = ∏ P(wi|C)
   - 簡化計算,但可能損失詞序信息

3. **Laplace Smoothing**: (count + α) / (total + α|V|)
   - 避免零機率問題
   - α=1 稱為加一平滑

4. **對數技巧**: 使用 log 避免數值下溢
   - log P(C|D) = log P(C) + Σ log P(wi|C)

5. **實作驗證**: 與 sklearn 對比確保正確性

### 📊 優缺點分析

**優點**:
- ✅ 實作簡單,計算高效
- ✅ 對小數據集表現良好
- ✅ 可解釋性強 (可查看特徵機率)
- ✅ 多分類任務表現穩定

**缺點**:
- ❌ 條件獨立假設過強 (忽略詞序)
- ❌ 對特徵相關性敏感
- ❌ 無法捕捉複雜模式

### 📚 延伸閱讀

- [Naive Bayes 數學推導](https://en.wikipedia.org/wiki/Naive_Bayes_classifier)
- [sklearn MultinomialNB 文檔](https://scikit-learn.org/stable/modules/naive_bayes.html)
- [垃圾郵件分類案例](https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection)

### 🚀 下一節預告

**底層實作02: 從零打造 MLP (多層感知器)**
- 前向傳播實作
- 反向傳播推導
- 激活函數與權重初始化

---

**課程**: iSpan Python NLP Cookbooks v2
**講師**: Claude AI
**最後更新**: 2025-10-17